## Sistema de recomendacion

In [ ]:
# Importando librerias
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split

In [ ]:
#Cargar los datasets
df_sco = pd.read_csv('./Datasets/score_movies.csv')
df_plat = pd.read_csv('./Datasets/plataformas.csv')

In [ ]:
df_plat = df_plat[['movieId', 'title']]

In [ ]:
df_plat = df_plat.reset_index(drop=True)

In [ ]:
# Pasos previos

N_filas = 100000 # Limitamos el dataset a N_filas

reader = Reader(rating_scale=(1, 5))

data = Dataset.load_from_df(df_sco[['userId', 'movieId', 'score']][:N_filas], reader)

# Separamos nuestros datos

trainset, testset = train_test_split(data, test_size=.25)

In [ ]:
# Entreno el modelo SVD

modelo = SVD()

modelo.fit(trainset)

In [ ]:
# Predigo

predictions = modelo.test(testset)

In [ ]:
predictions[1]

In [ ]:
# Hacemos una predicción al azar para usuario y película

modelo.predict(154,'as680')

In [ ]:
#Elijo un usuario al azar y hago una recomendación completa
usuario = 35487
rating = 4   # Tomamos películas a las que haya calificado con 4 o 5 estrellas
df_user = df_sco[(df_sco.userId == usuario) & (df_sco.score >= rating)]
df_user = df_user.reset_index(drop=True)

df_user = pd.merge(df_user, df_plat[['movieId', 'title']], on='movieId', how='left')
df_user

In [ ]:
recomendaciones_usuario = df_plat.iloc[:4499].copy()
print(recomendaciones_usuario.shape)
recomendaciones_usuario.head()

In [ ]:
#extraigo las películas que ya ha visto

usuario_vistas = df_sco[df_sco['userId'] == usuario]
print(usuario_vistas.shape)
usuario_vistas.head()

In [ ]:
# Eliminamos las peliculas que el usuario ya vió

recomendaciones_usuario = recomendaciones_usuario[~recomendaciones_usuario['movieId'].isin(usuario_vistas['movieId'])]

recomendaciones_usuario.shape

In [ ]:
# Recomiendo

recomendaciones_usuario['Estimate_Score'] = recomendaciones_usuario['movieId'].apply(lambda x: modelo.predict(usuario, x).est)

In [ ]:
recomendaciones_usuario = recomendaciones_usuario.sort_values('Estimate_Score', ascending=False)
print(recomendaciones_usuario.head(10))

## Predicción con el usuario y (____)

In [ ]:
def movie_recommendation(userId, movieId):
    # Obtenemos la predicción de la calificación que el usuario le daría a la película
    prediction = modelo.predict(userId, str(movieId))

    # Si la predicción es mayor o igual a 3.5, se recomienda la película
    if prediction.est >= 3.5:
        return "Recomendada", prediction.est
    else:
        return "No recomendada", prediction.est

In [ ]:
#Pruebo la función
movie_recommendation(59, 'ns65')

In [ ]:
#Evaluo el modelo
from surprise import accuracy

accuracy.rmse(predictions)

In [ ]:
#Optimizo hiperparametros
from surprise.model_selection import cross_validate
import numpy as np

rmse_test_means = []
factores = [1,2,4,8,16,32,64,128]

for factor in factores:
    print(factor)
    model = SVD(n_factors=factor)
    cv = cross_validate(model, data, measures=['RMSE'], cv = 3, verbose=True)
    rmse_test_means.append(np.mean(cv['test_rmse']))

## Crear una interfaz para el modelo

In [ ]:
#Creo una función para extraer el titulo
def titulo(movieId):
    return df_plat[df_plat.movieId == movieId].title.iloc[0].title()

In [ ]:
#Pruebo la función
titulo("ds568")

In [ ]:
#fusiono ambas funciono
def title_recommendation(userId, movieId):
    recomendado, puntaje = movie_recommendation(userId, movieId)
    titulo_1 = titulo(movieId)
    return (recomendado, puntaje,titulo_1)

In [ ]:
title_recommendation(548,"ds568")

In [ ]:
import gradio as gr

title = str("El tinder de tus peliculas")

with gr.Blocks(title=title) as demo:
    nro_usuario = gr.inputs.Number(label='tu numero de usario')
    peli = gr.Textbox(label='numero de pelicula')
    recomendacion = gr.Button('matchea con tu pelicula')
    title = gr.Textbox(label='tu match se llama')
    output = gr.Textbox(label= 'le gustas o no')
    score = gr.Textbox(label='calidad del match')
    recomendacion.click(fn = title_recommendation, inputs=[nro_usuario,peli], outputs=[output, score,title])
demo.launch(share = True)